# PrApp analysis

## Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tableone import TableOne
from statsmodels.stats.multitest import multipletests
from joblib import dump, load
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import log_loss
from kneefinder import KneeFinder
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import resample
import matplotlib.patches as mpatches
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Data

In [ ]:
# read preprocessed PrApp Data 
PrEP = pd.read_csv('data/PrEP_filtered.csv') 
no_PrEP = pd.read_csv('data/no_PrEP_filtered.csv') 

## Filtering for PrEP-indication
### Non-PrEP users with PrEP-indication

**Filter criteria for PrEP-indication (at least one criterion met)**
- sexualized drug use in the last 6 months
- STI in the last 12 months (syphilis, gonorrhoea, chlamydia, hepatitis C)
- ≥ 2 sex partners in the last 6 months AND low condom use (0-50%)

Steps
- Total number of non-PrEP users (preprocessed): 1,420 
- Non-PrEP users with PrEP indication: 431 
- Non-PrEP users without PrEP indication: 989


In [ ]:
print(f'Number of non-PrEP users (preprocessed): {no_PrEP.shape[0]}')

no_PrEP_indication = no_PrEP.loc[(no_PrEP['sexualized_drug_use'] == 'Ja') |                                         # drugs during sex       
                                 ((no_PrEP['test_syp_freq'] == '1') | (no_PrEP['test_syp_freq']  == '2+')) |        # STI: Syphilis
                                 ((no_PrEP['test_go_freq'] == '1') | (no_PrEP['test_go_freq']  == '2+')) |          # STI: Gonorrhö 
                                 ((no_PrEP['test_chla_freq'] == '1') | (no_PrEP['test_chla_freq']  == '2+')) |      # STI: Chlamydien 
                                 (no_PrEP['test_hcv_freq'] == 1.0) |                                                # STI: Hepatitis 
                                 (((no_PrEP['sex_partner'] == '2-3') |                                              # 2 or more sex partners in the last 6 months 
                                   (no_PrEP['sex_partner'] == '4-5') | (no_PrEP['sex_partner'] == '6-10') |
                                   (no_PrEP['sex_partner'] == '11-20') | (no_PrEP['sex_partner'] == '>20')) 
                                  & ((no_PrEP['condom'] == '0%') |                                                  # AND low condom use (0-50%)
                                     (no_PrEP['condom'] == '25%') | (no_PrEP['condom'] == '50%')))]  
print(f'Number of non-PrEP users with PrEP-indication: {no_PrEP_indication.shape[0]}')

no_PrEP_remaining = no_PrEP.loc[~no_PrEP.index.isin(no_PrEP_indication.index)] 
print(f'Number of non-PrEP users without PrEP-indication: {no_PrEP_remaining.shape[0]}')

### PrEP users with PrEP-indication

**Filter criteria for PrEP-indication (at least one criterion met)**
- drugs during sex in the last 6 months
- STI in the last 12 months (syphilis, gonorrhoea, chlamydia, hepatitis C)
- ≥ 2 sex partners in the last 6 months 

Steps
- Total number of PrEP users (preprocessed): 1,061 
- PrEP users with PrEP-indication: 1,027 
- PrEP users without PrEP-indication: 34

In [ ]:
print(f'Number of PrEP users (preprocessed): {PrEP.shape[0]}')

PrEP_indication = PrEP.loc[(PrEP['sexualized_drug_use'] == 'Ja') |                                                  # drugs during sex 
                           ((PrEP['test_syp_freq'] == '1') | (PrEP['test_syp_freq']  == '2+')) |                    # STI: Syphilis
                           ((PrEP['test_go_freq'] == '1') | (PrEP['test_go_freq']  == '2+')) |                      # STI: Gonorrhö 
                           ((PrEP['test_chla_freq'] == '1') | (PrEP['test_chla_freq']  == '2+')) |                  # STI: Chlamydien 
                           (PrEP['test_hcv_freq'] == 1.0) |                                                         # STI: Hepatitis 
                           ((PrEP['sex_partner'] == '2-3') |                                                        # 2 or more sex partners in the last 6 months
                            (PrEP['sex_partner'] == '4-5') | (PrEP['sex_partner'] == '6-10') |
                            (PrEP['sex_partner'] == '11-20') | (PrEP['sex_partner'] == '>20'))]    
print(f'Number of PrEP users with PrEP-indication: {PrEP_indication.shape[0]}')

PrEP_remaining = PrEP.loc[~PrEP.index.isin(PrEP_indication.index)] 
print(f'Number of PrEP users without PrEP-indication: {PrEP_remaining.shape[0]}')

### Stratification by survey wave 

- Non-PrEP users with PrEP-indication: 431 (wave 3: 349 and wave 4: 82)
- Non-PrEP users without PrEP-indication: 989 (wave 3: 772 and wave 4: 217)

- PrEP users with PrEP-indication: 1,027 (wave 3: 759 and wave 4: 268)
- PrEP users without PrEP-indication: 34 (wave 3: 22 and wave 4: 12)

In [ ]:
print("Non-PrEP users with a PrEP-indication, in wave 3:", len(no_PrEP_indication[no_PrEP_indication['welle'] == 3.0]))
print("Non-PrEP users with a PrEP-indication, in wave 4:", len(no_PrEP_indication[no_PrEP_indication['welle'] == 4.0]))

print("\nNon-PrEP users without a PrEP-indication, in wave 3:", len(no_PrEP_remaining[no_PrEP_remaining['welle'] == 3.0]))
print("Non-PrEP users without a PrEP-indication, in wave 4:", len(no_PrEP_remaining[no_PrEP_remaining['welle'] == 4.0]))

print("\nPrEP users with a PrEP-indication, in wave 3:", len(PrEP_indication[PrEP_indication['welle'] == 3.0]))
print("PrEP users with a PrEP-indication, in wave 4:", len(PrEP_indication[PrEP_indication['welle'] == 4.0]))

print("\nPrEP users without a PrEP-indication, in wave 3:", len(PrEP_remaining[PrEP_remaining['welle'] == 3.0]))
print("PrEP users without a PrEP-indication, in wave 4:", len(PrEP_remaining[PrEP_remaining['welle'] == 4.0]))

### Subgroups in non-PrEP users with PrEP-indication 
**Two subgroups in non-PrEP users with PrEP-indication (n = 431)**
- PrEP-indication induced by sexualized drug use: 130
- PrEP-indication induced by sexual behaviour (two or more sexual partner AND low condom use) and/or STI: 364

no exclusive groups: 63 participants are in both groups (have a PrEP-indication due to sexual behaviour/STI and sexualized drug use)

In [ ]:
# Non-PrEP users with a PrEP-indication due to their sexualized drug use in the last 6 months (n = 130)
no_PrEP_drug = no_PrEP_indication.loc[(no_PrEP_indication['sexualized_drug_use'] == 'Ja')] 
print(f'Number of PrEP users with PrEP-indication due to their sexualized drug use in the last 6 months: {no_PrEP_drug.shape[0]}')

# Non-PrEP users with a PrEP-indication due to a STI in the last 12 months (n = 143)
no_PrEP_STI = no_PrEP_indication.loc[((no_PrEP['test_syp_freq'] == '1') | (no_PrEP['test_syp_freq']  == '2+')) |        # STI: Syphilis
                                      ((no_PrEP['test_go_freq'] == '1') | (no_PrEP['test_go_freq']  == '2+')) |         # STI: Gonorrhö 
                                      ((no_PrEP['test_chla_freq'] == '1') | (no_PrEP['test_chla_freq']  == '2+')) |     # STI: Chlamydien 
                                      (no_PrEP['test_hcv_freq'] == 1.0) ]                                               # STI: Hepatitis C
print(f'Number of non-PrEP users with PrEP-indication due to a STI in the last 12 months: {no_PrEP_STI.shape[0]}')

# Non-PrEP users with a PrEP-indication due to their sexual behaviour (n = 266)
# sexual behaviour: two or more sexual partners in the last 6 months and low condom use (0-50%)
no_PrEP_sex = no_PrEP_indication.loc[(((no_PrEP_indication['sex_partner'] == '2-3') | (no_PrEP_indication['sex_partner'] == '4-5') | 
                                       (no_PrEP_indication['sex_partner'] == '6-10') | (no_PrEP_indication['sex_partner'] == '11-20') | (no_PrEP_indication['sex_partner'] == '>20')) &
                                       ((no_PrEP_indication['condom'] == '0%') | (no_PrEP_indication['condom'] == '25%') | (no_PrEP_indication['condom'] == '50%')))]  
print(f'Number of non-PrEP users with PrEP-indication due to their sexual behaviour in the last 6 months: {no_PrEP_sex.shape[0]}')

# Non-PrEP users with a PrEP-indication due to their sexual behaviour and/or STI history (n = 364)
no_PrEP_STI_and_sex = no_PrEP_indication.loc[(((no_PrEP['test_syp_freq'] == '1') | (no_PrEP['test_syp_freq']  == '2+')) |        
                                              ((no_PrEP['test_go_freq'] == '1') | (no_PrEP['test_go_freq']  == '2+')) |           
                                              ((no_PrEP['test_chla_freq'] == '1') |  (no_PrEP['test_chla_freq']  == '2+')) | (no_PrEP['test_hcv_freq'] == 1.0)) |      
                                             (((no_PrEP_indication['sex_partner'] == '2-3') | (no_PrEP_indication['sex_partner'] == '4-5') | 
                                               (no_PrEP_indication['sex_partner'] == '6-10') |(no_PrEP_indication['sex_partner'] == '11-20') | (no_PrEP_indication['sex_partner'] == '>20')) &
                                               ((no_PrEP_indication['condom'] == '0%') | (no_PrEP_indication['condom'] == '25%') | (no_PrEP_indication['condom'] == '50%')))]  
print(f'Number of non-PrEP users with PrEP-indication due to their sexual behaviour and/or STI history: {no_PrEP_STI_and_sex.shape[0]}')


# Non-PrEP users with a PrEP-indication due to their sexual behaviour and/or STI history AND sexualized drug use (n = 63)
common_indices = no_PrEP_STI_and_sex.index.intersection(no_PrEP_drug.index)
common_cases = no_PrEP_STI_and_sex.loc[common_indices] # 63 common participants
print(f'Number of non-PrEP users with PrEP-indication due to their sexual behaviour and/or STI history AND sexualized drug use: {common_cases.shape[0]}')

**Exclusive subgroups in non-PrEP users with PrEP-indication**
- PrEP-indication exclusively induced by sexualized drug use: 67
- PrEP-indication exclusively induced by sexual behaviour and/or STI: 301

In [ ]:
no_PrEP_drug.loc[:, 'INDICATION_SUBGROUP'] = 'sexualized drug-use'
no_PrEP_STI_and_sex.loc[:, 'INDICATION_SUBGROUP'] = 'sexual behaviour or STI'

# add 'both' as subgroup (both sexualized drug use and STI history and/or sexual behaviour)
no_PrEP_drug_both = no_PrEP_drug.copy()
no_PrEP_drug_both.loc[no_PrEP_drug_both.index.isin(common_cases.index), 'INDICATION_SUBGROUP'] = 'both' # 63 common participants with 'both'
no_PrEP_STI_and_sex_both = no_PrEP_STI_and_sex.copy()
no_PrEP_STI_and_sex_both.loc[no_PrEP_STI_and_sex_both.index.isin(common_cases.index), 'INDICATION_SUBGROUP'] = 'both' # 63 common participants with 'both'

# exclusive subgroups (without common cases)
no_PrEP_drug_exclusive = no_PrEP_drug.copy()
no_PrEP_drug_exclusive = no_PrEP_drug_exclusive[~no_PrEP_drug_exclusive.index.isin(common_cases.index)]
no_PrEP_STI_and_sex_exclusive = no_PrEP_STI_and_sex.copy()
no_PrEP_STI_and_sex_exclusive = no_PrEP_STI_and_sex_exclusive[~no_PrEP_STI_and_sex_exclusive.index.isin(common_cases.index)]

print(f'Non-PrEP users with a PrEP-indication exclusively due to sexualized drug use: {len(no_PrEP_drug_exclusive)}')
print(f'Non-PrEP users with a PrEP-indication exclusively due to sexual behaviour and/or STI history: {len(no_PrEP_STI_and_sex_exclusive)}')

## Baseline summary

In [ ]:
# concatenate PrEP user and non-PrEP users with a PrEP-indication for descriptive summary
concatenated_all_indication = pd.concat([PrEP_indication, no_PrEP_indication], ignore_index = True)

### Non-PrEP users vs. PrEP users by PrEP-indication criteria 


In [ ]:
data_filter_criteria = concatenated_all_indication[['sexualized_drug_use', 'test_syp_freq', 'test_go_freq', 'test_chla_freq','test_hcv_freq', 'condom', 'sex_partner','status']].copy() # PrEP-indication criteria

data_filter_criteria = data_filter_criteria.astype('category')
columns_filter_criteria = ['sexualized_drug_use', 'test_syp_freq', 'test_go_freq', 'test_chla_freq', 'test_hcv_freq', 'condom', 'sex_partner']
categorical_filter_criteria = ['sexualized_drug_use', 'test_syp_freq', 'test_go_freq', 'test_chla_freq', 'test_hcv_freq', 'condom', 'sex_partner']

null_value = 'None'
for col in categorical_filter_criteria:
    data_filter_criteria[col] = data_filter_criteria[col].cat.add_categories(null_value)

rename_filter_criteria = {'test_syp_freq': 'Syphilis diagnosis in the last 12 months',
                          'test_go_freq': 'Gonorrhoea diagnosis in the last 12 months', 
                          'test_chla_freq': 'Chlamydia diagnosis in the last 12 months', 
                          'test_hcv_freq': 'Hepatitis C diagnosis in the last 12 months', 
                          'sexualized_drug_use': 'Drugs during sex in the last 6 months',
                          'condom': 'Condom use',
                          'sex_partner': 'Sex partners in the last 6 months'}
table_filter_criteria = TableOne(data_filter_criteria, columns = columns_filter_criteria, 
                                 categorical = categorical_filter_criteria, rename = rename_filter_criteria, 
                                 groupby = 'status', pval = False, missing = False) 

print(table_filter_criteria.tabulate(tablefmt = "fancy grid")) 

### Non-PrEP users vs. PrEP users (baseline characteristics)

In [ ]:
data_table_baseline_summary = concatenated_all_indication.copy()

columns_baseline_summary  = ['age', 'landbin', 'URBAN_RURAL_AREA', 'HIV_SPECIALIST_DENSITY', 'ABSCHLUSS_TYPE', 'NET_EQUIVALENT_INCOME',
                             'SEXLIFE','sex_partner', 'condom', 'sex_freq','sexualized_drug_use', 'drugs_inject', 'sexwork', 'test_syp_freq', 'test_go_freq', 'test_chla_freq', 'test_hcv_freq',
                             'sex_with_men','sex_with_women', 'sex_with_nonbinary', 'sti_symptom', 'positive_test_syph', 'positive_test_go', 'positive_test_chla','positive_test_hcv', 'no_positive_test']
                            

categorical_baseline_summary  =  ['ABSCHLUSS_TYPE', 'SEXLIFE', 'age', 'sex_with_men', 'sex_with_women', 'sex_with_nonbinary', 'sex_partner', 'sex_freq', 'sexualized_drug_use', 'drugs_inject', 'sexwork',
                                  'condom', 'sti_symptom', 'positive_test_syph', 'positive_test_go', 'positive_test_chla', 'positive_test_hcv', 'no_positive_test', 'test_syp_freq',
                                  'test_go_freq', 'test_chla_freq', 'test_hcv_freq', 'landbin', 'NET_EQUIVALENT_INCOME', 'URBAN_RURAL_AREA', 'HIV_SPECIALIST_DENSITY']


rename_baseline_summary = {'age': 'Age', 
                           'landbin': 'Country of origin', 
                           'sex_partner': 'Sexpartners in the last 6 months',
                           'condom': 'Condom use',
                           'ABSCHLUSS_TYPE': 'General school qualification',
                           'sex_freq': 'Number of sexual intercourses in the last month',
                           'URBAN_RURAL_AREA': 'Urban vs. rural area',
                           'HIV_SPECIALIST_DENSITY': 'HIV-specialists density in the federal state of residence',
                           'NET_EQUIVALENT_INCOME': 'Monthly net equivalent income',
                           'SEXLIFE': 'Satisfaction with sex life'}

table_baseline_summary = TableOne(data_table_baseline_summary, columns = columns_baseline_summary, categorical = categorical_baseline_summary, 
                                       rename = rename_baseline_summary, pval = False, groupby = 'status', missing = False)

print(table_baseline_summary.tabulate(tablefmt = "fancy grid"))

## Reasons for not taking PrEP
Reasons for not taking PrEP stratified by survey wave, by PrEP-indication and by low-vs-high HIV-specialists density

Test: Chi-Squared (or Fisher's exact)


#### stratified by low-vs-high HIV-specialists density

In [ ]:
# group HIV-specialists density into low (0-9 HIV-specialists per 10,000 gay men) and high (10-13 HIV-specialists per 10,000 gay men)
no_PrEP_indication['HIV_SPECIALIST_DENSITY_grouped'] = np.where(
    no_PrEP_indication['HIV_SPECIALIST_DENSITY'].isin(["0-2", "3-5", "6-9"]), '0-9',
    np.where(no_PrEP_indication['HIV_SPECIALIST_DENSITY'] == "10-13", '10-13', 'NaN'))

no_PrEP_indication['HIV_SPECIALIST_DENSITY_grouped'] = np.where(
    no_PrEP_indication['HIV_SPECIALIST_DENSITY'].isna(), 'NaN', no_PrEP_indication['HIV_SPECIALIST_DENSITY_grouped'])

no_PrEP_indication['HIV_SPECIALIST_DENSITY_grouped'] = no_PrEP_indication['HIV_SPECIALIST_DENSITY_grouped'].replace('NaN', np.nan)

In [ ]:
data_reasons = no_PrEP_indication[['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons','reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                                  'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects',
                                  'welle', 'URBAN_RURAL_AREA', 'HIV_SPECIALIST_DENSITY_grouped']].copy()

columns_reasons = ['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons', 'reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                   'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects']

categorical_reasons = ['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons','reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                       'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects']

rename_reasons = {'reason_no_prep_low_HIV_risk':  'Personal HIV risk too low', 
                  'reason_no_prep_medical_reasons': 'Medical reasons', 
                  'reason_no_prep_no_doctor': 'No doctor prescribing PrEP',
                  'reason_no_prep_talking_sexlife_doctor': 'Not discussing my sex life with my doctor',
                  'reason_no_prep_effort_too_high': 'Too much effort: doctor visits + regular tests',
                  'reason_no_prep_daily_pill': 'Daily pill too strenuous', 
                  'reason_no_prep_fear_reactions': 'Fear: negative reactions from others/sex partners', 
                  'reason_no_prep_fear_side_effects': 'Fear: side effects', 'welle': 'survey wave'}

In [ ]:
# no correction for multiple testing 
table_reason_vs_density = TableOne(data_reasons, columns = columns_reasons , categorical = categorical_reasons, 
                   rename = rename_reasons, groupby = 'HIV_SPECIALIST_DENSITY_grouped', pval = True, htest_name = True, missing=False)

print(table_reason_vs_density.tabulate(tablefmt = "fancy grid")) 

In [ ]:
# correction for multiple testing 
table_reason_vs_density.htest_table['P-value_corrected'] = multipletests(table_reason_vs_density.htest_table['P-Value'].values.flatten(), method = 'fdr_bh')[1]
table_reason_vs_density.htest_table

### stratified by PrEP indication subgroups

In [ ]:
concatenated_no_PrEP_indication_subgroups_exclusive = pd.concat([no_PrEP_drug_exclusive, no_PrEP_STI_and_sex_exclusive], ignore_index = True) # exclusive PrEP-indication 
concatenated_no_PrEP_indication_subgroups_both = pd.concat([no_PrEP_drug_both, no_PrEP_STI_and_sex_exclusive], ignore_index = True) # includes 'both' PrEP-indication 

#### exclusive groups

In [ ]:
# exclusive groups 
data_reasons_subgroups = concatenated_no_PrEP_indication_subgroups_exclusive[['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons', 'reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                                                                              'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects','INDICATION_SUBGROUP']].copy()


columns_reasons_subgroups = ['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons','reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                             'reason_no_prep_effort_too_high','reason_no_prep_daily_pill','reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects']
categorical_reasons_subgroups = ['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons','reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                                 'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions', 'reason_no_prep_fear_side_effects']

rename_reasons_subgroups = {'reason_no_prep_low_HIV_risk':  'Personal HIV risk too low', 
                            'reason_no_prep_medical_reasons': 'Medical reasons', 
                            'reason_no_prep_no_doctor': 'No doctor prescribing PrEP',
                            'reason_no_prep_talking_sexlife_doctor': 'Not discussing my sex life with my doctor',
                            'reason_no_prep_effort_too_high': 'Too much effort: doctor visits + regular tests',
                            'reason_no_prep_daily_pill': 'Daily pill too strenuous', 
                            'reason_no_prep_fear_reactions': 'Fear: negative reactions from others/sex partners', 
                            'reason_no_prep_fear_side_effects': 'Fear: side effects', 'welle': 'survey wave'}


table_reason_vs_subgroup = TableOne(data_reasons_subgroups, columns = columns_reasons_subgroups , categorical = categorical_reasons_subgroups, 
                                    rename = rename_reasons_subgroups, groupby = 'INDICATION_SUBGROUP', pval = True, htest_name = True, overall = False)

print(table_reason_vs_subgroup.tabulate(tablefmt = "fancy grid")) 

In [ ]:
table_reason_vs_subgroup.htest_table['P-value_corrected'] = multipletests(table_reason_vs_subgroup.htest_table['P-Value'].values.flatten(), method = 'fdr_bh')[1]
table_reason_vs_subgroup.htest_table

#### including both

In [ ]:
# exclusive + both groups 
data_reasons_subgroups = concatenated_no_PrEP_indication_subgroups_both[['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons', 'reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                                                                         'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects','INDICATION_SUBGROUP']].copy()

columns_reasons_subgroups = ['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons', 'reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                             'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects']
categorical_reasons_subgroups  = ['reason_no_prep_low_HIV_risk','reason_no_prep_medical_reasons', 'reason_no_prep_no_doctor','reason_no_prep_talking_sexlife_doctor',
                                  'reason_no_prep_effort_too_high','reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions','reason_no_prep_fear_side_effects']

rename_reasons_subgroups = {'reason_no_prep_low_HIV_risk':  'Personal HIV risk too low', 
                            'reason_no_prep_medical_reasons': 'Medical reasons', 
                            'reason_no_prep_no_doctor': 'No doctor prescribing PrEP',
                            'reason_no_prep_talking_sexlife_doctor': 'Not discussing my sex life with my doctor',
                            'reason_no_prep_effort_too_high': 'Too much effort: doctor visits + regular tests',
                            'reason_no_prep_daily_pill': 'Daily pill too strenuous', 
                            'reason_no_prep_fear_reactions': 'Fear: negative reactions from others/sex partners', 
                            'reason_no_prep_fear_side_effects': 'Fear: side effects', 'welle': 'survey wave'}

table_reason_vs_subgroup = TableOne(data_reasons_subgroups, columns = columns_reasons_subgroups , categorical = categorical_reasons_subgroups, 
                                    rename = rename_reasons_subgroups, groupby = 'INDICATION_SUBGROUP')

print(table_reason_vs_subgroup.tabulate(tablefmt = "fancy grid")) 

## Encoding 

In [ ]:
# encode target variable
# 0: no PrEP, 1: PrEP
PrEP_indication['Target'] = 1 
no_PrEP_indication['Target'] = 0 

# concatenate PrEP and no PrEP (n = 1,458)
concatenated_population = pd.concat([PrEP_indication, no_PrEP_indication], ignore_index = True) 

# drop unnecessary columns (for multivariable analysis)
concatenated_population.drop(['welle', 'status', 'reason_no_prep_low_HIV_risk', 'reason_no_prep_medical_reasons', 'reason_no_prep_no_doctor', 'reason_no_prep_talking_sexlife_doctor',
                              'reason_no_prep_effort_too_high', 'reason_no_prep_daily_pill', 'reason_no_prep_fear_reactions', 'reason_no_prep_fear_side_effects', 'HIV_SPECIALIST_DENSITY_grouped', 'gendersex'], axis = 1, inplace = True) 

print(f'Number of Variables (including the target variable): {len(concatenated_population.columns)}')

In [ ]:
# binary variables --> transform into 0/1 
concatenated_population.replace(to_replace = {'ja': 1, 'Ja': 1, 'nein': 0, 'Nein': 0}, inplace=True) 

# binary w/o missing values (9 variables) --> no further encoding 
binary_columns = ['sex_with_men','sex_with_women','sex_with_nonbinary', 'positive_test_syph', 'positive_test_go','positive_test_chla','no_positive_test', 'positive_test_hcv', 'landbin']

# binary variables w/ missing values (4 variables) --> replace NaN values with 0 
already_dummy = ['test_hcv_freq', 'sti_symptom', 'sexualized_drug_use', 'sexwork']

for col in already_dummy:
    concatenated_population[col] = concatenated_population[col].fillna(0)
    
# categorical variables (13 variables)
categorical_columns = ['ABSCHLUSS_TYPE', 'SEXLIFE', 'age', 'sex_partner', 'sex_freq', 'drugs_inject', 'condom', 'NET_EQUIVALENT_INCOME', 'URBAN_RURAL_AREA', 'HIV_SPECIALIST_DENSITY', 'test_syp_freq', 'test_go_freq', 'test_chla_freq']

# encode categorical variables as dummy variables (ignores NaN values)
encoded_population = pd.get_dummies(data = concatenated_population, columns = categorical_columns, dtype = int, drop_first = False)

# dropping the reference category in the dummy encoded variables (most frequent category)
for col in categorical_columns:
    most_frequent = concatenated_population[col].value_counts().idxmax() 
    most_frequent_dummy_col = f"{col}_{most_frequent}"
    print(f'reference category for "{col}" is "{most_frequent_dummy_col}"')
    encoded_population.drop(most_frequent_dummy_col, axis = 1, inplace = True)

encoded_population.to_csv('data/PrApp_encoded.csv') 

In [ ]:
# table showing the encoding of the variables
summary_encoding = []

for col in concatenated_population.columns:
    value_counts = concatenated_population[col].value_counts(dropna=True)
    most_frequent = value_counts.idxmax()

    categories = []

    for cat in value_counts.index:
        label = str(cat) 
        
        if (len(value_counts.index) > 2): # only for categorical variables (and not binary variables)
            if cat == most_frequent: label += ' (R)'
        categories.append(label)

    summary_encoding.append({
            'Variable': col,
            'Categories': ', '.join(categories)})

summary_encoding_table = pd.DataFrame(summary_encoding)
summary_encoding_table 

## Hyperparameter tuning for different l1 ratios 
for l1 ratio is [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] \
find optimal regularisation strength using cross-validation 

CV: Repeated Stratified K-fold (stratified because of class imbalance)


Model: Logistic Regression
- elastic net with fixed l1-ratio 
- scoring: ROC AUC
- class_weights balanced (because of class imbalance)
- CV setting: 15-fold with 10 repeats + random state 1

In [ ]:
# encoded data
encoded_data = pd.read_csv('data/PrApp_encoded.csv', index_col=0) 

feature_cols = encoded_data.columns.drop('Target') 
feature_names = encoded_data[feature_cols].columns

X = encoded_data[feature_cols].values #  54 features
y = encoded_data['Target'].values # 0: No PrEP, 1: PrEP

In [ ]:
# function: hyperparameter tuning for each l1-ratio 
def parameter_tuning_ROC_AUC(X, y, l1_ratio, state, splits, repeats):
      
      rng = np.random.RandomState(state) # RandomState instances for models 
      cv_fold = RepeatedStratifiedKFold(n_splits = splits, n_repeats = repeats, random_state = state)

      regularisation_strengths = np.logspace(-1, -2, 100)
      logistic_regression_cv = LogisticRegressionCV(penalty = 'elasticnet', 
                                                   solver = 'saga', 
                                                   random_state = rng , 
                                                   max_iter = 1000, 
                                                   cv = cv_fold, 
                                                   l1_ratios = [l1_ratio], 
                                                   Cs = regularisation_strengths,
                                                   scoring ='roc_auc',
                                                   n_jobs = -1,
                                                   class_weight = "balanced", 
                                                   refit = True)
      logistic_regression_cv.fit(X, y)
      optimal_regulation_strength = logistic_regression_cv.C_[0]
      print(f'For l1 ratio = {l1_ratio}')
      print(f'the regulation strength with the highest AUC score: {optimal_regulation_strength}')
      
      mean_auc_scores = logistic_regression_cv.scores_[1].mean(axis = 0)
      mean_non_zero_coefficients = np.zeros(len(regularisation_strengths))

      results_AUC = []
      for C_index, C in enumerate(regularisation_strengths):
                  mean_ROC_AUC = mean_auc_scores[C_index][0]
                  
                  total_non_zero_coefs = 0
                  for fold_idx in range(cv_fold.get_n_splits()):
                        coefs = logistic_regression_cv.coefs_paths_[1][fold_idx][C_index]
                        total_non_zero_coefs += np.sum(coefs != 0)
                        
                  mean_non_zero_coefs = total_non_zero_coefs / cv_fold.get_n_splits()
                  results_AUC.append({
                        'C': C,
                        'l1_ratio': l1_ratio,
                        'mean_ROC_AUC': mean_ROC_AUC,
                        'mean_non_zero_coefficients': mean_non_zero_coefs})
      
      results_AUC  = pd.DataFrame(results_AUC)
      return logistic_regression_cv, results_AUC, optimal_regulation_strength



# function to get the results dataframe based on the saved logreg model
def parameter_tuning_results(logistic_regression_cv, l1_ratio, state, splits, repeats):
      
      rng = np.random.RandomState(state) # RandomState instances for models 
      cv_fold = RepeatedStratifiedKFold(n_splits = splits, n_repeats = repeats, random_state = state)

      regularisation_strengths = np.logspace(-1, -2, 100)
      optimal_regulation_strength = logistic_regression_cv.C_[0]
      mean_auc_scores = logistic_regression_cv.scores_[1].mean(axis = 0)
      mean_non_zero_coefficients = np.zeros(len(regularisation_strengths))
      results_AUC = []
      for C_index, C in enumerate(regularisation_strengths):
                  mean_ROC_AUC = mean_auc_scores[C_index][0]
                  
                  total_non_zero_coefs = 0
                  for fold_idx in range(cv_fold.get_n_splits()):
                        coefs = logistic_regression_cv.coefs_paths_[1][fold_idx][C_index]
                        total_non_zero_coefs += np.sum(coefs != 0)
                        
                  mean_non_zero_coefs = total_non_zero_coefs / cv_fold.get_n_splits()
                  results_AUC.append({
                        'C': C,
                        'l1_ratio': l1_ratio,
                        'mean_ROC_AUC': mean_ROC_AUC,
                        'mean_non_zero_coefficients': mean_non_zero_coefs})
      
      results_AUC  = pd.DataFrame(results_AUC)
      return results_AUC

def plot_cv_model(results_cv, l1_ratio, model):
      
      C = model.C_[0]

      regularisation_strengths = np.logspace(-1, -2, 100)

      alphas_elbow = (1 / regularisation_strengths)
      
      mean_coef = results_cv.loc[results_cv['mean_ROC_AUC'].idxmax()]['mean_non_zero_coefficients']
      mean_auc = results_cv.loc[results_cv['mean_ROC_AUC'].idxmax()]['mean_ROC_AUC']
      
      # plot: mean auc scores vs. regularization strength
      plt.figure(figsize=(6, 4))
      plt.plot(alphas_elbow , results_cv['mean_ROC_AUC'], 
               color = "royalblue" , linestyle='-')
      plt.xlabel('α (regularization strength)')
      plt.ylabel('Mean ROC AUC score')
      plt.title(f'Parameter tuning: mean AUC scores for different α (l1-ratio = {l1_ratio})')
      plt.show()
      
      print(f'Best AUC: \n'
      f'- Inverse of regularisation strength C: {C} \n'
      f'- Regularisation strength α: {1 / C} \n'
      f'- Mean no. non-zero coefficients: {mean_coef} \n'
      f'- Mean ROC AUC score: {mean_auc}')
      
      # plot: regularization path
      mean_coefs_value = np.mean(model.coefs_paths_[1][:], axis=(0))
      plt.figure(figsize=(6, 4))
      for i in range(mean_coefs_value.shape[2]):
            plt.plot(np.log10(alphas_elbow), 
            mean_coefs_value[:, 0, i])
      plt.xlabel('log(α)')
      plt.ylabel('Mean coefficient value')
      plt.title(f'Regularization Path: l1-ratio = {l1_ratio}')
      plt.show()
      
      if l1_ratio == 0.0: 
            plt.figure(figsize=(6, 4))
            plt.scatter(alphas_elbow, results_cv['mean_non_zero_coefficients'],
                  linewidths = 1, alpha = .7,
                  s = 50,
                  c = results_cv['mean_ROC_AUC'])
            plt.ylabel('Mean no. non-zero coefficients')
            plt.xlabel('α (regularization strength)')
            plt.colorbar(label = 'Mean ROC AUC score')
            plt.show()
            
            print('no elbow point')
      
      else:
            # plot: elbow plot 
            kf_auc_vs_coef  = KneeFinder(results_cv['mean_ROC_AUC'], results_cv['mean_non_zero_coefficients'])
            knee_x_auc_vs_coef, knee_y_auc_vs_coef = kf_auc_vs_coef.find_knee() 
      
            idx_point_coef_vs_auc =  np.where(results_cv['mean_ROC_AUC'].values == knee_x_auc_vs_coef)[0] 
            alpha_coef_vs_auc = alphas_elbow[idx_point_coef_vs_auc][0]

      
            plt.figure(figsize=(6, 4))
            plt.plot(results_cv['mean_ROC_AUC'], results_cv['mean_non_zero_coefficients'], 
                  color = "royalblue" , linestyle='-')
            plt.xlabel('Mean ROC AUC Score')
            plt.ylabel('Mean no. non-zero coefficients')
            plt.axvline(x = knee_x_auc_vs_coef, color='crimson', linestyle='--', label = f" elbow point ")
            plt.legend()
            plt.title(f'Mean ROC AUC l1-ratio = {l1_ratio}')
            plt.show()
      
            plt.figure(figsize=(6, 4))
            plt.scatter(alphas_elbow, results_cv['mean_non_zero_coefficients'],
                        linewidths = 1, alpha = .7, s = 50, c = results_cv['mean_ROC_AUC'])
            plt.ylabel('Mean no. non-zero coefficients')
            plt.xlabel('α (regularization strength)')
            plt.colorbar(label = 'Mean ROC AUC score')
            plt.axvline(x = alpha_coef_vs_auc , color='crimson', linestyle='--', 
                        label = f"elbow point at α = {round(alpha_coef_vs_auc , 2)}")
            plt.legend()
            plt.show()
            print(f'Elbow point: \n'
                  f'- Inverse of regularisation strength C: {1 / alpha_coef_vs_auc} \n'
                  f'- Regularisation strength α: {alpha_coef_vs_auc} \n'
                  f'- Mean no. non-zero coefficients: {knee_x_auc_vs_coef} \n'
                  f'- Mean ROC AUC score: {knee_y_auc_vs_coef}')
            
            return alpha_coef_vs_auc

### l1-ratio = 0 (L2 regularization)

elbow point: not existing because the mean number of mean non-zero coefficients is constant

In [ ]:
logreg_0, results_0, optimal_C_0 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.0, state = 1, splits = 15, repeats = 10)
dump(logreg_0, 'results/parameter_tuning_models/logreg_0.joblib') 

logreg_0 = load('results/parameter_tuning_models/logreg_0.joblib') 
results_0 = parameter_tuning_results(logreg_0, l1_ratio = 0.0, state = 1, splits = 15, repeats = 10) 
plot_cv_model(results_0, l1_ratio = 0.0, model = logreg_0)

### l1-ratio = 0.1

In [ ]:
logreg_0_1, results_0_1, optimal_C_0_1 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.1, state = 1, splits = 15, repeats = 10)
dump(logreg_0_1, 'results/parameter_tuning_models/logreg_0_1.joblib')

logreg_0_1 = load('results/parameter_tuning_models/logreg_0_1.joblib')
results_0_1 = parameter_tuning_results(logreg_0_1, l1_ratio = 0.1, state = 1, splits = 15, repeats = 10)
alpha_elbow_0_1 = plot_cv_model(results_0_1, l1_ratio = 0.1, model = logreg_0_1)

### l1-ratio = 0.2

In [ ]:
logreg_0_2, results_0_2, optimal_C_0_2 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.2, state = 1, splits = 15, repeats = 10)
dump(logreg_0_2, 'results/parameter_tuning_models/logreg_0_2.joblib') 

logreg_0_2 = load('results/parameter_tuning_models/logreg_0_2.joblib') 
results_0_2 = parameter_tuning_results(logreg_0_2, l1_ratio = 0.2, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_2 = plot_cv_model(results_0_2, l1_ratio = 0.2, model = logreg_0_2)

### l1-ratio = 0.3

In [ ]:
logreg_0_3, results_0_3, optimal_C_0_3 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.3, state = 1, splits = 15, repeats = 10)
dump(logreg_0_3, 'results/parameter_tuning_models/logreg_0_3.joblib')

logreg_0_3 = load('results/parameter_tuning_models/logreg_0_3.joblib') 
results_0_3 = parameter_tuning_results(logreg_0_3, l1_ratio = 0.3, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_3 = plot_cv_model(results_0_3, l1_ratio = 0.3, model = logreg_0_3)

### l1-ratio = 0.4

In [ ]:
logreg_0_4, results_0_4, optimal_C_0_4 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.4, 
                                                                                    state = 1, splits = 15, repeats = 10)
dump(logreg_0_4, 'results/parameter_tuning_models/logreg_0_4.joblib')

logreg_0_4 = load('results/parameter_tuning_models/logreg_0_4.joblib') 
results_0_4 = parameter_tuning_results(logreg_0_4, l1_ratio = 0.4, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_4 = plot_cv_model(results_0_4, l1_ratio = 0.4, model = logreg_0_4)

### l1-ratio = 0.5

In [ ]:
logreg_0_5, results_0_5, optimal_C_0_5 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.5, state = 1, splits = 15, repeats = 10)
dump(logreg_0_5, 'results/parameter_tuning_models/logreg_0_5.joblib')

logreg_0_5 = load('results/parameter_tuning_models/logreg_0_5.joblib') 
results_0_5 = parameter_tuning_results(logreg_0_5, l1_ratio = 0.5, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_5 = plot_cv_model(results_0_5, l1_ratio = 0.5, model = logreg_0_5)

### l1-ratio = 0.6

In [ ]:
logreg_0_6, results_0_6, optimal_C_0_6 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.6, 
                                                                                    state = 1, splits = 15, repeats = 10)
dump(logreg_0_6, 'results/parameter_tuning_models/logreg_0_6.joblib') 

logreg_0_6 = load('results/parameter_tuning_models/logreg_0_6.joblib') 
results_0_6 = parameter_tuning_results(logreg_0_6, l1_ratio = 0.6, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_6 = plot_cv_model(results_0_6, l1_ratio = 0.6, model = logreg_0_6)

### l1-ratio = 0.7

In [ ]:
logreg_0_7, results_0_7, optimal_C_0_7 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.7, 
                                                                                    state = 1, splits = 15, repeats = 10)
dump(logreg_0_7, 'results/parameter_tuning_models/logreg_0_7.joblib')

logreg_0_7 = load('results/parameter_tuning_models/logreg_0_7.joblib') 
results_0_7 = parameter_tuning_results(logreg_0_7, l1_ratio = 0.7, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_7 = plot_cv_model(results_0_7, l1_ratio = 0.7, model = logreg_0_7)

### l1-ratio = 0.8

In [ ]:
logreg_0_8, results_0_8, optimal_C_0_8 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.8, 
                                                                                    state = 1, splits = 15, repeats = 10)
dump(logreg_0_8, 'results/parameter_tuning_models/logreg_0_8.joblib')

logreg_0_8 = load('results/parameter_tuning_models/logreg_0_8.joblib') 
results_0_8 = parameter_tuning_results(logreg_0_8, l1_ratio = 0.8, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_8 = plot_cv_model(results_0_8, l1_ratio = 0.8, model = logreg_0_8)

### l1-ratio = 0.9

In [ ]:
logreg_0_9, results_0_9, optimal_C_0_9 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 0.9, 
                                                                                    state = 1, splits = 15, repeats = 10)
dump(logreg_0_9, 'results/parameter_tuning_models/logreg_0_9.joblib')

logreg_0_9 = load('results/parameter_tuning_models/logreg_0_9.joblib') 
results_0_9 = parameter_tuning_results(logreg_0_9, l1_ratio = 0.9, state = 1, splits = 15, repeats = 10) 
alpha_elbow_0_9 = plot_cv_model(results_0_9, l1_ratio = 0.9, model = logreg_0_9)

### l1-ratio = 1.0

In [ ]:
logreg_1_0, results_1_0, optimal_C_1_0 = parameter_tuning_ROC_AUC(X, y, l1_ratio = 1.0, 
                                                                                    state = 1, splits = 15, repeats = 10)
dump(logreg_1_0, 'results/parameter_tuning_models/logreg_1_0.joblib')

logreg_1_0 = load('results/parameter_tuning_models/logreg_1_0.joblib') 
results_1_0 = parameter_tuning_results(logreg_1_0, l1_ratio = 1.0, state = 1, splits = 15, repeats = 10) 
alpha_elbow_1_0 = plot_cv_model(results_1_0, l1_ratio = 1.0, model = logreg_1_0)

## Model selection

for each l1_ratio a model is fitted with the regularisation strength from elbow point 

### models

In [ ]:
rng = np.random.RandomState(1)

# models one for each l1-ratio fitted with regularisation strength from elbow point 
model_0_1 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_1,
            l1_ratio = 0.1,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver ='saga')
      
model_0_1.fit(X, y)

dump(model_0_1, 'results/parameter_tuning_models/logreg_0_1_elbow.joblib') 

model_0_2 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_2,
            l1_ratio = 0.2,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_2.fit(X, y)
dump(model_0_2, 'results/parameter_tuning_models/logreg_0_2_elbow.joblib')

model_0_3 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_3,
            l1_ratio = 0.3,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_3.fit(X, y)
dump(model_0_3, 'results/parameter_tuning_models/logreg_0_3_elbow.joblib')

model_0_4 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_4,
            l1_ratio = 0.4,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_4.fit(X, y)
dump(model_0_4, 'results/parameter_tuning_models/logreg_0_4_elbow.joblib')

model_0_5 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_5,
            l1_ratio = 0.5,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_5.fit(X, y)
dump(model_0_5, 'results/parameter_tuning_models/logreg_0_5_elbow.joblib')

model_0_6 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_6,
            l1_ratio = 0.6,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_6.fit(X, y)
dump(model_0_6, 'results/parameter_tuning_models/logreg_0_6_elbow.joblib')

model_0_7 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_7,
            l1_ratio = 0.7,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_7.fit(X, y)
dump(model_0_7, 'results/parameter_tuning_models/logreg_0_7_elbow.joblib')

model_0_8 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_8,
            l1_ratio = 0.8,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_8.fit(X, y)
dump(model_0_8, 'results/parameter_tuning_models/logreg_0_8_elbow.joblib')

model_0_9 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_0_9,
            l1_ratio = 0.9,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_0_9.fit(X, y)
dump(model_0_9, 'results/parameter_tuning_models/logreg_0_9_elbow.joblib')

model_1_0 = LogisticRegression(
            penalty= 'elasticnet',
            C = 1 / alpha_elbow_1_0,
            l1_ratio = 1.0,
            max_iter = 1000,
            class_weight = "balanced",
            random_state = rng, 
            solver='saga')
      
model_1_0.fit(X, y)
dump(model_1_0, 'results/parameter_tuning_models/logreg_1_0_elbow.joblib')

In [ ]:
# models one for each l1-ratio fitted with regularisation strength from elbow point
model_0_1 = load('results/parameter_tuning_models/logreg_0_1_elbow.joblib') 
model_0_2 = load('results/parameter_tuning_models/logreg_0_2_elbow.joblib') 
model_0_3 = load('results/parameter_tuning_models/logreg_0_3_elbow.joblib') 
model_0_4 = load('results/parameter_tuning_models/logreg_0_4_elbow.joblib') 
model_0_5 = load('results/parameter_tuning_models/logreg_0_5_elbow.joblib') 
model_0_6 = load('results/parameter_tuning_models/logreg_0_6_elbow.joblib') 
model_0_7 = load('results/parameter_tuning_models/logreg_0_7_elbow.joblib') 
model_0_8 = load('results/parameter_tuning_models/logreg_0_8_elbow.joblib') 
model_0_9 = load('results/parameter_tuning_models/logreg_0_9_elbow.joblib') 
model_1_0 = load('results/parameter_tuning_models/logreg_1_0_elbow.joblib') 

### selection

In [ ]:
def calculate_aic(y_true, y_pred_proba, num_params):
    # log_loss: negative log-likelihood of a logistic model (- log_loss: log-likelihood) ln(L)
    # normalize = False: sum of the per-sample losses
    log_like = - log_loss(y_true, y_pred_proba, normalize = False)
    aic = 2 * num_params - 2 * log_like
    return aic

def calculate_bic(y_true, y_pred_proba, num_params):
    log_like = - log_loss(y_true, y_pred_proba, normalize = False)
    n = len(y_true) # no. observations
    bic = - 2 * log_like + np.log(n) * num_params 
    return bic


def model_ICs(model, l1, optimal_C, modus):
    y_pred_proba = model.predict_proba(X)[:, 1]
    num_params = len(model.coef_[0]) + 1 # + intercept 
    aic = calculate_aic(y, y_pred_proba, num_params)
    bic = calculate_bic(y, y_pred_proba, num_params)
    
    c = model.coef_.ravel()
    non_zero_index = np.nonzero(c)[0]
    non_zero_coefficients = len(non_zero_index)
    
    auc = roc_auc_score(y, y_pred_proba)
    
    results_IC = pd.DataFrame({
        'l1-ratio': [l1],
        'modus': [modus],
        'inverse regularisation strength C': [optimal_C],
        'regularisation strength alpha': [1/optimal_C],
        'ROC_AUC': [auc],
        'non_zero_coefficients': [non_zero_coefficients],
        'AIC': [aic],
        'BIC': [bic]
    })
      
    return results_IC

In [ ]:
# fitted models for each l1-ratio with regularisation strength from elbow point detection
results_best_elbow_0_1 = model_ICs(model_0_1, l1 = 0.1, optimal_C = 1 / alpha_elbow_0_1, modus = "elbow point")
results_best_elbow_0_2 = model_ICs(model_0_2, l1 = 0.2, optimal_C = 1 / alpha_elbow_0_2, modus = "elbow point")
results_best_elbow_0_3 = model_ICs(model_0_3, l1 = 0.3, optimal_C = 1 / alpha_elbow_0_3, modus = "elbow point")
results_best_elbow_0_4 = model_ICs(model_0_4, l1 = 0.4, optimal_C = 1 / alpha_elbow_0_4, modus = "elbow point")
results_best_elbow_0_5 = model_ICs(model_0_5, l1 = 0.5, optimal_C = 1 / alpha_elbow_0_5, modus = "elbow point")
results_best_elbow_0_6 = model_ICs(model_0_6, l1 = 0.6, optimal_C = 1 / alpha_elbow_0_6, modus = "elbow point")
results_best_elbow_0_7 = model_ICs(model_0_7, l1 = 0.7, optimal_C = 1 / alpha_elbow_0_7, modus = "elbow point")
results_best_elbow_0_8 = model_ICs(model_0_8, l1 = 0.8, optimal_C = 1 / alpha_elbow_0_8, modus = "elbow point")
results_best_elbow_0_9 = model_ICs(model_0_9, l1 = 0.9, optimal_C = 1 / alpha_elbow_0_9, modus = "elbow point")
results_best_elbow_1_0 = model_ICs(model_1_0, l1 = 1.0, optimal_C = 1 / alpha_elbow_1_0, modus = "elbow point")

results_model_selection_best_elbow = pd.concat([results_best_elbow_0_1, results_best_elbow_0_2, 
                                                results_best_elbow_0_3, results_best_elbow_0_4, results_best_elbow_0_5, results_best_elbow_0_6, results_best_elbow_0_7,
                                                results_best_elbow_0_8, results_best_elbow_0_9, results_best_elbow_1_0], axis=0, ignore_index=True)  

results_model_selection_best_elbow_rounded = results_model_selection_best_elbow.round(2)
results_model_selection_best_elbow_rounded               

In [ ]:
# scatter plot: AIC vs. sparsity (number of non-zero coefficients)
# we want: low AIC and low no. of  -> based on your data decide which model to select

# trained models with regularisation strength from elbow point
plt.figure(figsize=(8, 4))
scatter_elbow = sns.scatterplot(data = results_model_selection_best_elbow, x ='non_zero_coefficients', 
                                y ='AIC', hue ='l1-ratio', palette = sns.color_palette("magma", 10), s = 50)
for i in range(results_model_selection_best_elbow.shape[0]): # add labels to scatter points 
    plt.text(
        results_model_selection_best_elbow['non_zero_coefficients'].iloc[i] + 0.6, 
        results_model_selection_best_elbow['AIC'].iloc[i] - 4,
        f'{results_model_selection_best_elbow["l1-ratio"].iloc[i]:.1f}',  
        fontsize = 10,
        ha = 'left')
handles, labels = scatter_elbow.get_legend_handles_labels() # add alpha to legend

custom_labels = [
    f'l1-ratio: {results_model_selection_best_elbow["l1-ratio"].iloc[i]:.1f}, '
    f' α: {results_model_selection_best_elbow["regularisation strength alpha"].iloc[i]:.2f}'
    for i in range(len(labels) )
]

plt.legend(handles = handles[:], labels = custom_labels, title = 'Model parameters', title_fontsize='12', loc='center left',  bbox_to_anchor=(1, 0.5))
plt.xlabel('Number of non-zero coefficients')
plt.ylabel('AIC')
plt.title('Model selection: AIC vs. sparsity')
plt.show()

## Multivariable analysis

### Multivariable regression model
selected model from model selection

In [ ]:
final_model = load('results/parameter_tuning_models/logreg_0_4_elbow.joblib')  # based on model selection 
final_model.fit(X, y)

# final coefficients
final_coefficients = final_model.coef_
final_non_zero_coefficients = final_coefficients[np.nonzero(final_coefficients)]
final_non_zero_feature_names = feature_names[np.nonzero(final_coefficients)[1]]
final_non_zero_coefficients_df = pd.DataFrame({'Feature': final_non_zero_feature_names,
                                               'Coefficient': final_non_zero_coefficients})

final_coefficients_df = pd.DataFrame({'Feature': feature_names,
                                      'Coefficient': final_coefficients[0]})

# optimal classification threshold 
y_proba = final_model.predict_proba(X)[:, 1]
fpr, tpr, thresholds = roc_curve(y, y_proba)
optimal_threshold_index  = np.argmax(tpr  - fpr)
optimal_threshold  = thresholds[optimal_threshold_index]
y_pred_adjusted = (y_proba >= optimal_threshold).astype(int)

#### non-zero regression coefficients (scatterplot)

In [ ]:
# scatterplot: non-zero coefficients
plt.figure(figsize=(8, 10))
plt.scatter(final_non_zero_coefficients_df['Coefficient'], final_non_zero_coefficients_df['Feature'], color='b')
plt.xlabel('Regression coefficient')
plt.axvline(0, color='black', linestyle='--')
plt.ylabel('Features')
plt.title('Final Model: Non-zero Regression Coefficients')

for i, row in final_non_zero_coefficients_df.iterrows():
    plt.text(
        x=row['Coefficient'] + 0.03 * max(abs(final_non_zero_coefficients_df['Coefficient'])), 
        y=row['Feature'],
        s=f"{row['Coefficient']:.2f}",
        va='center',
        fontsize=10,
        color='black'
    )
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# non-zero coefficients
round(final_non_zero_coefficients_df,2)

#### optimal classification threshold

In [ ]:
# ROC curve with optimal classification threshold
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, label='ROC Curve', color = 'black')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.scatter(fpr[optimal_threshold_index], tpr[optimal_threshold_index], 
            marker='o', color='red', label=f'Optimal threshold: {optimal_threshold:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
adjusted_classification_threshold = pd.DataFrame({
    'True Label': y,
    'Predicted Probability': y_proba,
    'Group': np.where(y == 1, 'PrEP', 'no PrEP')})

colors_group = {'PrEP': 'seagreen', 'no PrEP': 'orange'}

# violin plot: predicted probabilities with optimal classification threshold
plt.figure(figsize=(5, 5))
sns.violinplot(x = 'True Label', y ='Predicted Probability', data = adjusted_classification_threshold, palette = colors_group, hue = 'Group')
plt.axhline(optimal_threshold, color='red', linestyle='--', label=f'optimal threshold: {optimal_threshold:.2f}')
plt.xlabel('True label')
plt.xticks([0, 1], labels = ['no PrEP', 'PrEP'])
plt.ylabel('Predicted probability')
plt.legend()
plt.show()

### Bootstrapping 
10,000 stratified bootstrapped sampled 

In [ ]:
final_model = load('results/parameter_tuning_models/logreg_0_4_elbow.joblib') # load final selected model 
rng = np.random.RandomState(1)
bootstrapped_coefficients = []

for _ in range(10000):
    X_bootstrap, y_bootstrap = resample(X, y, 
                                        random_state = rng, 
                                        stratify = y,
                                        replace = True)
    final_model.fit(X_bootstrap, y_bootstrap)
    bootstrapped_coefficients.append(final_model.coef_[0])
    
bootstrapped_coefficients_df = pd.DataFrame(bootstrapped_coefficients, columns = feature_names) # all bootstrapped regression coefficients
non_zero_bootstrapped_coefficients_df = bootstrapped_coefficients_df.loc[:, (bootstrapped_coefficients_df != 0).any(axis=0)] # all bootstrapped non-zeroregression coefficients

#### non-parametric bootstrap: percentile confidence interval 

In [ ]:
CI_lower = np.percentile(np.array(bootstrapped_coefficients_df), 2.5, axis = 0) # lower 95% confidence interval
CI_upper = np.percentile(np.array(bootstrapped_coefficients_df), 97.5, axis = 0) # upper 95% confidence interval

multivariable_coef_results = final_coefficients_df.copy()
multivariable_coef_results['lower 95% CI'] = CI_lower
multivariable_coef_results['upper 95% CI'] = CI_upper 

In [ ]:
significant_coefs = (CI_lower > 0) | (CI_upper < 0) # significant coefficients based on confidence intervals (check: is 0 included in the CI)

significant_from_CI_df = pd.DataFrame({
    'Feature': bootstrapped_coefficients_df.columns,
    'Lower 95% CI': CI_lower,
    'Upper 95% CI': CI_upper,
    'Significant': significant_coefs
})

significant_from_CI_df.loc[significant_from_CI_df['Significant'] == True]

#### non-parametric bootstrap: hypothesis test 

H0: β = 0 

H1: β ≠ 0 

In [ ]:
bootstrap_samples = 10000
p_values_features = {}

for feature in bootstrapped_coefficients_df.columns:
    bst_coefs = bootstrapped_coefficients_df[feature] # bootstrapped coefficients for the feature

    greater_than_zero = np.sum(bst_coefs >= 0) / bootstrap_samples # sum of bootstrapped coefficients >= zero / bootstrap samples
    less_than_zero = np.sum(bst_coefs <= 0) / bootstrap_samples # sum of bootstrapped coefficients <= zero / bootstrap samples
    
    p_value = min(greater_than_zero, less_than_zero)
    p_values_features[feature] = p_value


p_values_features_df = pd.DataFrame([p_values_features]).T
p_values_features_df.rename(columns = {0: 'p_values'}, inplace = True)


multivariable_coef_results['p_values'] = p_values_features_df['p_values'].values


p_values_features_df.sort_values(by = 'p_values', ascending = True)

In [ ]:
columns_2_decimal = ['Coefficient', 'lower 95% CI', 'upper 95% CI']
columns_3_decimal = ['p_values']

multivariable_coef_results_rounded = multivariable_coef_results.copy()
multivariable_coef_results_rounded[columns_2_decimal] = multivariable_coef_results_rounded[columns_2_decimal].round(2)
multivariable_coef_results_rounded[columns_3_decimal] = multivariable_coef_results_rounded[columns_3_decimal].round(3)

multivariable_coef_results_rounded.to_csv('results/multivariable_analysis_result.csv', index = True)

multivariable_coef_results_rounded

#### boxplot significant variables 

In [ ]:
# 13 significant features
significant_features_multivariable = multivariable_coef_results.loc[multivariable_coef_results['p_values'] < 0.05]['Feature']
significant_bst_coef = bootstrapped_coefficients_df[significant_features_multivariable]

In [ ]:
plot_variables = ['sexualized_drug_use', 'positive_test_chla', 'SEXLIFE_Discontent', 'SEXLIFE_Sex doesnt matter right now',
                  'age_18-29 yrs', 'age_40-49 yrs', 'age_50-80 yrs', 'sex_partner_0', 'sex_partner_1', 'sex_partner_11-20',
                  'sex_partner_2-3', 'sex_partner_4-5', 'sex_partner_6-10',
                  'condom_0%', 'condom_50%', 'condom_75%', 'condom_>95%',
                  'NET_EQUIVALENT_INCOME_1 bis <1000', 'NET_EQUIVALENT_INCOME_1000 bis <2000', 'NET_EQUIVALENT_INCOME_3000 bis <4000',
                  'NET_EQUIVALENT_INCOME_4000 bis <5000', 'NET_EQUIVALENT_INCOME_5000+',
                  'HIV_SPECIALIST_DENSITY_0-2', 'HIV_SPECIALIST_DENSITY_10-13', 'HIV_SPECIALIST_DENSITY_3-5']

plot_bst_strata = bootstrapped_coefficients_df[plot_variables]

# add reference categories as 0 
plot_bst_strata['30-39 years (R)'] = 0
plot_bst_strata['6-9 (R)'] = 0
plot_bst_strata['2000 - <3000€ (R)'] = 0
plot_bst_strata['Content (R)'] = 0
plot_bst_strata[r'$\geq$ 20 (R)'] = 0
plot_bst_strata['25% (R)'] = 0

plot_bst_strata_renamed = plot_bst_strata.rename(columns = {'age_18-29 yrs': r'$\mathbf{Age}$ 18-29 years',
                                                            'age_40-49 yrs': '40-49 years', 'age_50-80 yrs': '50-80 years',
                                                            'HIV_SPECIALIST_DENSITY_0-2': r'$\mathbf{HIV-specialists\ density}$ 0-2', 
                                                            'HIV_SPECIALIST_DENSITY_3-5': '3-5', 'HIV_SPECIALIST_DENSITY_10-13': '10-13',
                                                            'NET_EQUIVALENT_INCOME_1 bis <1000': r'$\mathbf{Monthly\ net-equivalent\ income}$ 1 - <1000€',
                                                            'NET_EQUIVALENT_INCOME_1000 bis <2000': '1000 - <2000€',
                                                            'NET_EQUIVALENT_INCOME_3000 bis <4000': '3000 - <4000€',
                                                            'NET_EQUIVALENT_INCOME_4000 bis <5000': '4000 - <5000€', 
                                                            'NET_EQUIVALENT_INCOME_5000+': r'$\geq$ 5000€',
                                                            'SEXLIFE_Discontent': r'$\mathbf{Satisfaction\ with\ sex\ life}$ Discontent',
                                                            'SEXLIFE_Sex doesnt matter right now': "Sex doesn't matter right now",
                                                            'sex_partner_0': r'$\mathbf{Sex\ partners\ in\ the\ last\ 6\ months}$ 0',
                                                            'sex_partner_1': '1',
                                                            'sex_partner_2-3': '2-3',
                                                            'sex_partner_4-5': '4-5',
                                                            'sex_partner_6-10': '6-10',
                                                            'sex_partner_11-20': '11-20',
                                                            'condom_0%': r'$\mathbf{Condom\ use}$ 0%',
                                                            'condom_50%': '50%',
                                                            'condom_75%': '75%',
                                                            'condom_>95%': '>95%',
                                                            'sexualized_drug_use': r'$\mathbf{Sexualized\ substance\ use\ in\ the\ last\ 6\ months}$',
                                                            'positive_test_chla': r'$\mathbf{Chlamydia\ diagnosis\ in\ the\ past}$'})

column_order = [r'$\mathbf{Age}$ 18-29 years', '30-39 years (R)', '40-49 years', '50-80 years', r'$\mathbf{HIV-specialists\ density}$ 0-2', '3-5', '6-9 (R)', '10-13',
                r'$\mathbf{Monthly\ net-equivalent\ income}$ 1 - <1000€', '1000 - <2000€', '2000 - <3000€ (R)', '3000 - <4000€', '4000 - <5000€', r'$\geq$ 5000€',
                r'$\mathbf{Satisfaction\ with\ sex\ life}$ Discontent', 'Content (R)', "Sex doesn't matter right now", 
                r'$\mathbf{Sex\ partners\ in\ the\ last\ 6\ months}$ 0', '1', '2-3', '4-5', '6-10', '11-20', r'$\geq$ 20 (R)', 
                r'$\mathbf{Condom\ use}$ 0%', '25% (R)', '50%', '75%', '>95%',
                r'$\mathbf{Sexualized\ substance\ use\ in\ the\ last\ 6\ months}$', 
                r'$\mathbf{Chlamydia\ diagnosis\ in\ the\ past}$']

# color by significantly associated with PrEP use (green) / non-PrEP use (orange), or not significantly associated (grey)
column_colors = {r'$\mathbf{Age}$ 18-29 years': 'orange', '30-39 years (R)': 'grey', '40-49 years': 'grey', '50-80 years': 'grey',
                r'$\mathbf{HIV-specialists\ density}$ 0-2': 'grey', '3-5': 'grey', '6-9 (R)': 'grey', '10-13': 'seagreen',
                r'$\mathbf{Monthly\ net-equivalent\ income}$ 1 - <1000€': 'orange', '1000 - <2000€': 'grey', '2000 - <3000€ (R)': 'grey', '3000 - <4000€': 'grey', 
                '4000 - <5000€': 'grey', r'$\geq$ 5000€': 'grey',
                r'$\mathbf{Satisfaction\ with\ sex\ life}$ Discontent': 'orange', 'Content (R)': 'grey', "Sex doesn't matter right now": 'grey', 
                r'$\mathbf{Sex\ partners\ in\ the\ last\ 6\ months}$ 0': 'grey', '1': 'orange', '2-3': 'orange', '4-5': 'orange', '6-10': 'grey', '11-20': 'seagreen', r'$\geq$ 20 (R)': 'grey', 
                r'$\mathbf{Condom\ use}$ 0%': 'seagreen', '25% (R)': 'grey', '50%': 'orange', '75%': 'grey', '>95%': 'orange',
                r'$\mathbf{Sexualized\ substance\ use\ in\ the\ last\ 6\ months}$': 'orange',
                r'$\mathbf{Chlamydia\ diagnosis\ in\ the\ past}$': 'seagreen'}

color_palette = [column_colors.get(col, 'gray') for col in plot_bst_strata_renamed.columns]

In [ ]:
plt.figure(figsize=(6, 10))
sns.boxplot(data = plot_bst_strata_renamed , 
            orient = "h", 
            fliersize = 0.3,
            linewidth = 0.8,
            order = column_order,
            palette= color_palette)
plt.xlabel('Bootstrapped regression coefficients β')
plt.ylabel('')

plt.axvline(x = 0, color = 'grey', linestyle='--') # add vertical line at 0 to mark no effect 
no_prep = mpatches.Patch(color = 'orange', label = 'Non-PrEP use')
prep = mpatches.Patch(color = 'seagreen', label = 'PrEP use')
plt.legend(handles=[no_prep, prep])  
plt.show()

## Sensitivity analysis

### Missing value imputation 
check if missing value imputation is correct by encoding missing as a seperate category 

#### Encoding
same as in main analysis with the exception that for the categorical variables the missing values are treated as a seperate category and are encoded as well 

In [ ]:
# concatenate PrEP users and non-PrEP users with a PrEP indication (n = 1,458)
concatenated_population_sensitivity = pd.concat([PrEP_indication, no_PrEP_indication], ignore_index = True) 

# drop unnecessary columns (for multivariable analysis)
concatenated_population_sensitivity.drop(['welle', 'status', 'reason_no_prep_low_HIV_risk',
       'reason_no_prep_medical_reasons', 'reason_no_prep_no_doctor',
       'reason_no_prep_talking_sexlife_doctor',
       'reason_no_prep_effort_too_high', 'reason_no_prep_daily_pill',
       'reason_no_prep_fear_reactions', 'reason_no_prep_fear_side_effects',
       'HIV_SPECIALIST_DENSITY_grouped', 'gendersex'], axis = 1, inplace = True) 

print(f'Number of Variables (including the target variable): {len(concatenated_population_sensitivity.columns)}')


# already binary variables --> transform into 0/1 
concatenated_population_sensitivity.replace(to_replace = {'ja': 1, 'Ja': 1, 'nein': 0, 'Nein': 0}, inplace = True) 

# already binary w/o missing values --> no further encoding 
binary_columns = ['sex_with_men','sex_with_women','sex_with_nonbinary', 
                  'positive_test_syph', 'positive_test_go','positive_test_chla','no_positive_test',
                  'positive_test_hcv', 'landbin']

# categorical variables (including binary variables with NaNs) 
categorical_columns_sensitivity = ['ABSCHLUSS_TYPE', 'SEXLIFE', 'age', 'sex_partner', 'sex_freq', 
                                   'drugs_inject', 'condom',
                                   'NET_EQUIVALENT_INCOME', 'URBAN_RURAL_AREA', 'HIV_SPECIALIST_DENSITY', 
                                   'test_syp_freq', 'test_go_freq', 'test_chla_freq', 'test_hcv_freq', 'sti_symptom',
                                   'sexualized_drug_use', 'sexwork']

# encode categorical variables as dummy variables (keep NaNs as a seperate category)
encoded_population_sensitivity = pd.get_dummies(data = concatenated_population_sensitivity, 
                         columns = categorical_columns_sensitivity, 
                         dtype = int, dummy_na = True, 
                         drop_first = False)


# dropping the reference category in the dummy encoded variables (most frequent)
for col in categorical_columns_sensitivity:
    most_frequent = concatenated_population_sensitivity[col].value_counts(dropna = False).idxmax() 
    most_frequent_dummy_col = f"{col}_{most_frequent}"
    print(f'reference category for "{col}" is "{most_frequent_dummy_col}"')
    encoded_population_sensitivity.drop(most_frequent_dummy_col, axis = 1, inplace = True)

#### Multivariable analysis
multivariable regression with new encoded data (includes NaN values as a seperate category)


In [ ]:
feature_cols_nan = encoded_population_sensitivity.columns.drop('Target') 
feature_names_nan = encoded_population_sensitivity[feature_cols_nan].columns

X_nan = encoded_population_sensitivity[feature_cols_nan].values 
y_nan = encoded_population_sensitivity['Target'] 

# load final selected model (same model as in main multivariable analysis)
rng = np.random.RandomState(1)
final_model = load('results/parameter_tuning_models/logreg_0_4_elbow.joblib') 

final_model.fit(X_nan, y_nan)
sens_coef = final_model.coef_
sens_coef_non_zero = sens_coef[np.nonzero(sens_coef)]
sens_feature_names_non_zero = feature_names_nan[np.nonzero(sens_coef)[1]]
sens_coef_non_zero_df = pd.DataFrame({'Feature': sens_feature_names_non_zero,
                                   'Coefficient': sens_coef_non_zero})
sens_coef_df = pd.DataFrame({'Feature': feature_names_nan,
                             'Coefficient': sens_coef[0]})

##### Bootstrapping

In [ ]:
bootstrapped_coefficients_nan = []

for _ in range(10000):
    X_bst, y_bst = resample(X_nan, y_nan, 
                            random_state = rng, 
                            stratify = y_nan,
                            replace = True)
    final_model.fit(X_bst, y_bst)
    bootstrapped_coefficients_nan.append(final_model.coef_[0])
    
bst_coef_nan  = pd.DataFrame(bootstrapped_coefficients_nan, columns = feature_names_nan)
non_zero_bst_coef_nan = bst_coef_nan.loc[:, (bst_coef_nan  != 0).any(axis = 0)]

In [ ]:
# percentile confidence interval 
CI_lower_nan = np.percentile(np.array(bst_coef_nan), 2.5, axis = 0)
CI_upper_nan = np.percentile(np.array(bst_coef_nan), 97.5, axis = 0)

multivariable_coef_results_nan = sens_coef_df.copy()
multivariable_coef_results_nan['lower 95% CI'] = CI_lower_nan
multivariable_coef_results_nan['upper 95% CI'] = CI_upper_nan

In [ ]:
significant_coefs_nan = (CI_lower_nan > 0) | (CI_upper_nan < 0) # significant coefficients based on confidence intervals (check: is 0 included in the CI)

significant_from_CI_nan_df = pd.DataFrame({
    'Feature': feature_names_nan,
    'Lower 95% CI': CI_lower_nan,
    'Upper 95% CI': CI_upper_nan,
    'Significant': significant_coefs_nan
})

# compare this to significant based on bootstrapped p-value 
significant_from_CI_nan_df.loc[significant_from_CI_nan_df['Significant'] == True]

In [ ]:
bootstrap_samples = 10000
p_values_features_nan = {}

for feature in feature_names_nan:
    bst_coefs_nan = bst_coef_nan[feature] # bootstrapped coefficients for the feature

    greater_than_zero = np.sum(bst_coefs_nan >= 0) / bootstrap_samples # sum of bootstrapped coefficients >= zero / bootstrap samples
    less_than_zero = np.sum(bst_coefs_nan <= 0) / bootstrap_samples # sum of bootstrapped coefficients <= zero / bootstrap samples
    
    p_value_nan = min(greater_than_zero, less_than_zero)
    p_values_features_nan[feature] = p_value_nan


p_values_features_nan_df = pd.DataFrame([p_values_features_nan]).T
p_values_features_nan_df.rename(columns = {0: 'p_values'}, inplace = True)


multivariable_coef_results_nan['p_values'] = p_values_features_nan_df['p_values'].values

p_values_features_nan_df.sort_values(by = 'p_values', ascending = True).head(20)

In [ ]:
columns_2_decimal = ['Coefficient', 'lower 95% CI', 'upper 95% CI']
columns_3_decimal = ['p_values']

multivariable_coef_results_nan_rounded = multivariable_coef_results_nan.copy()
multivariable_coef_results_nan_rounded[columns_2_decimal] = multivariable_coef_results_nan_rounded[columns_2_decimal].round(2)
multivariable_coef_results_nan_rounded[columns_3_decimal] = multivariable_coef_results_nan_rounded[columns_3_decimal].round(3)

multivariable_coef_results_nan_rounded.to_csv('results/multivariable_analysis_result_sensitivity.csv', index = True)

multivariable_coef_results_nan_rounded

In [ ]:
# 14 significant features
significant_features_multivariable_nan = multivariable_coef_results_nan.loc[multivariable_coef_results_nan['p_values'] < 0.05]['Feature']
significant_bst_coef_nan = bst_coef_nan[significant_features_multivariable_nan]

In [ ]:
print(len(significant_features_multivariable_nan))

In [ ]:
# one additional significant feature in sensitivity analysis: SEXLIFE_nan
# rest of the significant features are the same as in the main multivariable analysis
significant_features_multivariable_nan 